# Fungsi AND OR NOT

In [29]:
import math
class BooleanModel():

        @staticmethod
        def and_operation(left_operand, right_operand):
            # perform 'merge'
            result = [] # results list to be returned
            l_index = 0 # current index in left_operand
            r_index = 0 # current index in right_operand
            l_skip = int(math.sqrt(len(left_operand)))
            # skip pointer distance for l_index
            r_skip = int(math.sqrt(len(right_operand)))
            # skip pointer distance for r_index

            while (l_index < len(left_operand) and r_index < len(right_operand)):
                l_item = left_operand[l_index]
                r_item = right_operand[r_index]

                # case 1: if match
                if (l_item == r_item):
                    result.append(l_item) # add to results
                    l_index += 1 # advance left index
                    r_index += 1 # advance right index

                # case 2: if left item is more than right item
                elif (l_item > r_item):
                    # if r_index can be skipped (if new r_index is still within range and resulting item is <= left item)
                    if (r_index + r_skip < len(right_operand)) and right_operand[r_index + r_skip] <= l_item:
                        r_index += r_skip
                    # else advance r_index by 1
                    else:
                        r_index += 1
                
                # case 3: if left item is less than right item
                else:
                    # if l_index can be skipped (if new l_index is still within range and resulting item is <= right item)
                    if (l_index + l_skip < len(left_operand)) and left_operand[l_index + l_skip] <= r_item:
                        l_index += l_skip
                    # else advance l_index by 1
                    else:
                        l_index += 1
            return result

        @staticmethod
        def or_operation(left_operand, right_operand):
            result = [] # union of left and right operand
            l_index = 0 # current index in left_operand
            r_index = 0 # current index in right_operand
            # while lists have not yet been covered
            while (l_index < len(left_operand) or r_index < len(right_operand)):
                # if both list are not yet exhausted
                if (l_index < len(left_operand) and r_index < len(right_operand)):
                    l_item = left_operand[l_index] # current item in left_operand
                    r_item = right_operand[r_index] # current item in right_operand
                    
                    # case 1: if items are equal, add either one to result and advance both pointers
                    if (l_item == r_item):
                        result.append(l_item)
                        l_index += 1
                        r_index += 1

                    # case 2: l_item greater than r_item, add r_item and advance r_index
                    elif (l_item > r_item):
                        result.append(r_item)
                        r_index += 1

                    # case 3: l_item lower than r_item, add l_item and advance l_index
                    else:
                        result.append(l_item)
                        l_index += 1

                    # if left_operand list is exhausted, append r_item and advance r_index
                elif (l_index >= len(left_operand)):
                    r_item = right_operand[r_index]
                    result.append(r_item)
                    r_index += 1
                # else if right_operand list is exhausted, append l_item and advance l_index
                else:
                    l_item = left_operand[l_index]
                    result.append(l_item)
                    l_index += 1
            return result

        
        @staticmethod
        def not_operation(right_operand, indexed_docIDs):
            # complement of an empty list is list of all indexed docIDs
            if (not right_operand):
                return indexed_docIDs
            result = []
            r_index = 0 # index for right operand
            for item in indexed_docIDs:
                # if item do not match that in right_operand, it belongs to compliment
                if (item != right_operand[r_index]):
                    result.append(item)
                # else if item matches and r_index still can progress, advance it by 1
                elif (r_index + 1 < len(right_operand)):
                    r_index += 1
            



# Fungsi Untuk Mengurutkan Prioritas Query

In [30]:
def parse_query(infix_tokens):
    """ Parse Query
    Parsing done using Shunting Yard Algorithm
    """
    precedence = {}
    precedence['NOT'] = 3
    precedence['AND'] = 2
    precedence['OR'] = 1
    precedence['('] = 0
    precedence[')'] = 0
    
    output = []
    operator_stack = []
    
    for token in infix_tokens:
        if (token == '('):
            operator_stack.append(token)
        # if right bracket, pop all operators from operator stack onto output until we hit left bracket
        elif (token == ')'):
            operator = operator_stack.pop()
            while operator != '(':
                output.append(operator)
                operator = operator_stack.pop()
        # if operator, pop operators from operator stack to queue if they are of higher precedence
        elif (token in precedence):
            # if operator stack is not empty
            if (operator_stack):
                current_operator = operator_stack[-1]
                while (operator_stack and precedence[current_operator] > precedence[token]):
                    output.append(operator_stack.pop())
                    if (operator_stack):
                        current_operator = operator_stack[-1]
            operator_stack.append(token) # add token to stack
        else:
            output.append(token.lower())
    # while there are still operators on the stack, pop them into the queue
    while (operator_stack):
        output.append(operator_stack.pop())
    return output

# Boolean Retrieval Model

In [31]:
import collections

def process_query(query, n_docs, inverted_index):
    # prepare query list
    query = query.replace('(', '( ')
    query = query.replace(')', ' )')
    query = query.split(' ')
    print(query)

    indexed_docIDs = list(range(1, n_docs + 1))
    results_stack = []
    postfix_queue = collections.deque(parse_query(query)) # get query in postfix notation as a queue

    while postfix_queue:
        token = postfix_queue.popleft()
        result = [] # the evaluated result at each stage
        # if operand, add postings list for term to results stack
        if (token != 'AND' and token != 'OR' and token != 'NOT'):
            token = stemmer.stem(token) # stem the token
            # default empty list if not in dictionary
            if (token in inverted_index):
                result = inverted_index[token]

        elif (token == 'AND'):
            right_operand = results_stack.pop()
            left_operand = results_stack.pop()
            result = BooleanModel.and_operation(left_operand, right_operand) # evaluate AND
            
        elif (token == 'OR'):
            right_operand = results_stack.pop()
            left_operand = results_stack.pop()
            result = BooleanModel.or_operation(left_operand, right_operand) # evaluate OR

        elif (token == 'NOT'):
            right_operand = results_stack.pop()
            result = BooleanModel.not_operation(right_operand, indexed_docIDs) # evaluate NOT
        results_stack.append(result)

    # NOTE: at this point results_stack should only have one item and it is the final result
    if len(results_stack) != 1:
        print("ERROR: Invalid Query. Please check query syntax.") #check for errors
        return None
    return results_stack.pop()

# # Preprocessing

In [42]:
import os
import re

from spacy.lang.id.stop_words import STOP_WORDS
from spacy.lang.id import Indonesian
import spacy

#menyiapkan stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#tetapkan path untuk berita
path = r"C:\Users\septi\Desktop\Desktop Folder\pelajaran stis\Tugas\Semester 5\Information Retrieval\berita"

#normalisasi
normal_list= {"@": "di", "abis": "habis", "ad": "ada", "adlh": "adalah", "afaik": "as far as i know", "ahaha": "haha", "aj": "saja", "ajep-ajep": "dunia gemerlap", "ak": "saya", "akika": "aku", "akkoh": "aku", "akuwh": "aku", "alay": "norak", "alow": "halo", "ambilin": "ambilkan", "ancur": "hancur", "anjrit": "anjing", "anter": "antar", "ap2": "apa-apa", "apasih": "apa sih", "apes": "sial", "aps": "apa", "aq": "saya", "aquwh": "aku", "asbun": "asal bunyi", "aseekk": "asyik", "asekk": "asyik", "asem": "asam", "aspal": "asli tetapi palsu", "astul": "asal tulis", "ato": "atau", "au ah": "tidak mau tahu", "awak": "saya", "ay": "sayang", "ayank": "sayang", "b4": "sebelum", "bakalan": "akan", "bandes": "bantuan desa", "bangedh": "banget", "banpol": "bantuan polisi", "banpur": "bantuan tempur", "basbang": "basi", "bcanda": "bercanda", "bdg": "bandung", "begajulan": "nakal", "beliin": "belikan", "bencong": "banci", "bentar": "sebentar", "ber3": "bertiga", "beresin": "membereskan", "bete": "bosan", "beud": "banget", "bg": "abang", "bgmn": "bagaimana", "bgt": "banget", "bijimane": "bagaimana", "bintal": "bimbingan mental", "bkl": "akan", "bknnya": "bukannya", "blegug": "bodoh", "blh": "boleh", "bln": "bulan", "blum": "belum", "bnci": "benci", "bnran": "yang benar", "bodor": "lucu", "bokap": "ayah", "boker": "buang air besar", "bokis": "bohong", "boljug": "boleh juga", "bonek": "bocah nekat", "boyeh": "boleh", "br": "baru", "brg": "bareng", "bro": "saudara laki-laki", "bru": "baru", "bs": "bisa", "bsen": "bosan", "bt": "buat", "btw": "ngomong-ngomong", "buaya": "tidak setia", "bubbu": "tidur", "bubu": "tidur", "bumil": "ibu hamil", "bw": "bawa", "bwt": "buat", "byk": "banyak", "byrin": "bayarkan", "cabal": "sabar", "cadas": "keren", "calo": "makelar", "can": "belum", "capcus": "pergi", "caper": "cari perhatian", "ce": "cewek", "cekal": "cegah tangkal", "cemen": "penakut", "cengengesan": "tertawa", "cepet": "cepat", "cew": "cewek", "chuyunk": "sayang", "cimeng": "ganja", "cipika cipiki": "cium pipi kanan cium pipi kiri", "ciyh": "sih", "ckepp": "cakep", "ckp": "cakep", "cmiiw": "correct me if i'm wrong", "cmpur": "campur", "cong": "banci", "conlok": "cinta lokasi", "cowwyy": "maaf", "cp": "siapa", "cpe": "capek", "cppe": "capek", "cucok": "cocok", "cuex": "cuek", "cumi": "Cuma miscall", "cups": "culun", "curanmor": "pencurian kendaraan bermotor", "curcol": "curahan hati colongan", "cwek": "cewek", "cyin": "cinta", "d": "di", "dah": "deh", "dapet": "dapat", "de": "adik", "dek": "adik", "demen": "suka", "deyh": "deh", "dgn": "dengan", "diancurin": "dihancurkan", "dimaafin": "dimaafkan", "dimintak": "diminta", "disono": "di sana", "dket": "dekat", "dkk": "dan kawan-kawan", "dll": "dan lain-lain", "dlu": "dulu", "dngn": "dengan", "dodol": "bodoh", "doku": "uang", "dongs": "dong", "dpt": "dapat", "dri": "dari", "drmn": "darimana", "drtd": "dari tadi", "dst": "dan seterusnya", "dtg": "datang", "duh": "aduh", "duren": "durian", "ed": "edisi", "egp": "emang gue pikirin", "eke": "aku", "elu": "kamu", "emangnya": "memangnya", "emng": "memang", "endak": "tidak", "enggak": "tidak", "envy": "iri", "ex": "mantan", "fax": "facsimile", "fifo": "first in first out", "folbek": "follow back", "fyi": "sebagai informasi", "gaada": "tidak ada uang", "gag": "tidak", "gaje": "tidak jelas", "gak papa": "tidak apa-apa", "gan": "juragan", "gaptek": "gagap teknologi", "gatek": "gagap teknologi", "gawe": "kerja", "gbs": "tidak bisa", "gebetan": "orang yang disuka", "geje": "tidak jelas", "gepeng": "gelandangan dan pengemis", "ghiy": "lagi", "gile": "gila", "gimana": "bagaimana", "gino": "gigi nongol", "githu": "gitu", "gj": "tidak jelas", "gmana": "bagaimana", "gn": "begini", "goblok": "bodoh", "golput": "golongan putih", "gowes": "mengayuh sepeda", "gpny": "tidak punya", "gr": "gede rasa", "gretongan": "gratisan", "gtau": "tidak tahu", "gua": "saya", "guoblok": "goblok", "gw": "saya", "ha": "tertawa", "haha": "tertawa", "hallow": "halo", "hankam": "pertahanan dan keamanan", "hehe": "he", "helo": "halo", "hey": "hai", "hlm": "halaman", "hny": "hanya", "hoax": "isu bohong", "hr": "hari", "hrus": "harus", "hubdar": "perhubungan darat", "huff": "mengeluh", "hum": "rumah", "humz": "rumah", "ilang": "hilang", "ilfil": "tidak suka", "imho": "in my humble opinion", "imoetz": "imut", "item": "hitam", "itungan": "hitungan", "iye": "iya", "ja": "saja", "jadiin": "jadi", "jaim": "jaga image", "jayus": "tidak lucu", "jdi": "jadi", "jem": "jam", "jga": "juga", "jgnkan": "jangankan", "jir": "anjing", "jln": "jalan", "jomblo": "tidak punya pacar", "jubir": "juru bicara", "jutek": "galak", "k": "ke", "kab": "kabupaten", "kabor": "kabur", "kacrut": "kacau", "kadiv": "kepala divisi", "kagak": "tidak", "kalo": "kalau", "kampret": "sialan", "kamtibmas": "keamanan dan ketertiban masyarakat", "kamuwh": "kamu", "kanwil": "kantor wilayah", "karna": "karena", "kasubbag": "kepala subbagian", "katrok": "kampungan", "kayanya": "kayaknya", "kbr": "kabar", "kdu": "harus", "kec": "kecamatan", "kejurnas": "kejuaraan nasional", "kekeuh": "keras kepala", "kel": "kelurahan", "kemaren": "kemarin", "kepengen": "mau", "kepingin": "mau", "kepsek": "kepala sekolah", "kesbang": "kesatuan bangsa", "kesra": "kesejahteraan rakyat", "ketrima": "diterima", "kgiatan": "kegiatan", "kibul": "bohong", "kimpoi": "kawin", "kl": "kalau", "klianz": "kalian", "kloter": "kelompok terbang", "klw": "kalau", "km": "kamu", "kmps": "kampus", "kmrn": "kemarin", "knal": "kenal", "knp": "kenapa", "kodya": "kota madya", "komdis": "komisi disiplin", "komsov": "komunis sovyet", "kongkow": "kumpul bareng teman-teman", "kopdar": "kopi darat", "korup": "korupsi", "kpn": "kapan", "krenz": "keren", "krm": "kirim", "kt": "kita", "ktmu": "ketemu", "ktr": "kantor", "kuper": "kurang pergaulan", "kw": "imitasi", "kyk": "seperti", "la": "lah", "lam": "salam", "lamp": "lampiran", "lanud": "landasan udara", "latgab": "latihan gabungan", "lebay": "berlebihan", "leh": "boleh", "lelet": "lambat", "lemot": "lambat", "lgi": "lagi", "lgsg": "langsung", "liat": "lihat", "litbang": "penelitian dan pengembangan", "lmyn": "lumayan", "lo": "kamu", "loe": "kamu", "lola": "lambat berfikir", "louph": "cinta", "low": "kalau", "lp": "lupa", "luber": "langsung, umum, bebas, dan rahasia", "luchuw": "lucu", "lum": "belum", "luthu": "lucu", "lwn": "lawan", "maacih": "terima kasih", "mabal": "bolos", "macem": "macam", "macih": "masih", "maem": "makan", "magabut": "makan gaji buta", "maho": "homo", "mak jang": "kaget", "maksain": "memaksa", "malem": "malam", "mam": "makan", "maneh": "kamu", "maniez": "manis", "mao": "mau", "masukin": "masukkan", "melu": "ikut", "mepet": "dekat sekali", "mgu": "minggu", "migas": "minyak dan gas bumi", "mikol": "minuman beralkohol", "miras": "minuman keras", "mlah": "malah", "mngkn": "mungkin", "mo": "mau", "mokad": "mati", "moso": "masa", "mpe": "sampai", "msk": "masuk", "mslh": "masalah", "mt": "makan teman", "mubes": "musyawarah besar", "mulu": "melulu", "mumpung": "selagi", "munas": "musyawarah nasional", "muntaber": "muntah dan berak", "musti": "mesti", "muupz": "maaf", "mw": "now watching", "n": "dan", "nanam": "menanam", "nanya": "bertanya", "napa": "kenapa", "napi": "narapidana", "napza": "narkotika, alkohol, psikotropika, dan zat adiktif ", "narkoba": "narkotika, psikotropika, dan obat terlarang", "nasgor": "nasi goreng", "nda": "tidak", "ndiri": "sendiri", "ne": "ini", "nekolin": "neokolonialisme", "nembak": "menyatakan cinta", "ngabuburit": "menunggu berbuka puasa", "ngaku": "mengaku", "ngambil": "mengambil", "nganggur": "tidak punya pekerjaan", "ngapah": "kenapa", "ngaret": "terlambat", "ngasih": "memberikan", "ngebandel": "berbuat bandel", "ngegosip": "bergosip", "ngeklaim": "mengklaim", "ngeksis": "menjadi eksis", "ngeles": "berkilah", "ngelidur": "menggigau", "ngerampok": "merampok", "ngga": "tidak", "ngibul": "berbohong", "ngiler": "mau", "ngiri": "iri", "ngisiin": "mengisikan", "ngmng": "bicara", "ngomong": "bicara", "ngubek2": "mencari-cari", "ngurus": "mengurus", "nie": "ini", "nih": "ini", "niyh": "nih", "nmr": "nomor", "nntn": "nonton", "nobar": "nonton bareng", "np": "now playing", "ntar": "nanti", "ntn": "nonton", "numpuk": "bertumpuk", "nutupin": "menutupi", "nyari": "mencari", "nyekar": "menyekar", "nyicil": "mencicil", "nyoblos": "mencoblos", "nyokap": "ibu", "ogah": "tidak mau", "ol": "online", "ongkir": "ongkos kirim", "oot": "out of topic", "org2": "orang-orang", "ortu": "orang tua", "otda": "otonomi daerah", "otw": "on the way, sedang di jalan", "pacal": "pacar", "pake": "pakai", "pala": "kepala", "pansus": "panitia khusus", "parpol": "partai politik", "pasutri": "pasangan suami istri", "pd": "pada", "pede": "percaya diri", "pelatnas": "pemusatan latihan nasional", "pemda": "pemerintah daerah", "pemkot": "pemerintah kota", "pemred": "pemimpin redaksi", "penjas": "pendidikan jasmani", "perda": "peraturan daerah", "perhatiin": "perhatikan", "pesenan": "pesanan", "pgang": "pegang", "pi": "tapi", "pilkada": "pemilihan kepala daerah", "pisan": "sangat", "pk": "penjahat kelamin", "plg": "paling", "pmrnth": "pemerintah", "polantas": "polisi lalu lintas", "ponpes": "pondok pesantren", "pp": "pulang pergi", "prg": "pergi", "prnh": "pernah", "psen": "pesan", "pst": "pasti", "pswt": "pesawat", "pw": "posisi nyaman", "qmu": "kamu", "rakor": "rapat koordinasi", "ranmor": "kendaraan bermotor", "re": "reply", "ref": "referensi", "rehab": "rehabilitasi", "rempong": "sulit", "repp": "balas", "restik": "reserse narkotika", "rhs": "rahasia", "rmh": "rumah", "ru": "baru", "ruko": "rumah toko", "rusunawa": "rumah susun sewa", "ruz": "terus", "saia": "saya", "salting": "salah tingkah", "sampe": "sampai", "samsek": "sama sekali", "sapose": "siapa", "satpam": "satuan pengamanan", "sbb": "sebagai berikut", "sbh": "sebuah", "sbnrny": "sebenarnya", "scr": "secara", "sdgkn": "sedangkan", "sdkt": "sedikit", "se7": "setuju", "sebelas dua belas": "mirip", "sembako": "sembilan bahan pokok", "sempet": "sempat", "sendratari": "seni drama tari", "sgt": "sangat", "shg": "sehingga", "siech": "sih", "sikon": "situasi dan kondisi", "sinetron": "sinema elektronik", "siramin": "siramkan", "sj": "saja", "skalian": "sekalian", "sklh": "sekolah", "skt": "sakit", "slesai": "selesai", "sll": "selalu", "slma": "selama", "slsai": "selesai", "smpt": "sempat", "smw": "semua", "sndiri": "sendiri", "soljum": "sholat jumat", "songong": "sombong", "sory": "maaf", "sosek": "sosial-ekonomi", "sotoy": "sok tahu", "spa": "siapa", "sppa": "siapa", "spt": "seperti", "srtfkt": "sertifikat", "stiap": "setiap", "stlh": "setelah", "suk": "masuk", "sumpek": "sempit", "syg": "sayang", "t4": "tempat", "tajir": "kaya", "tau": "tahu", "taw": "tahu", "td": "tadi", "tdk": "tidak", "teh": "kakak perempuan", "telat": "terlambat", "telmi": "telat berpikir", "temen": "teman", "tengil": "menyebalkan", "tepar": "terkapar", "tggu": "tunggu", "tgu": "tunggu", "thankz": "terima kasih", "thn": "tahun", "tilang": "bukti pelanggaran", "tipiwan": "TvOne", "tks": "terima kasih", "tlp": "telepon", "tls": "tulis", "tmbah": "tambah", "tmen2": "teman-teman", "tmpah": "tumpah", "tmpt": "tempat", "tngu": "tunggu", "tnyta": "ternyata", "tokai": "tai", "toserba": "toko serba ada", "tpi": "tapi", "trdhulu": "terdahulu", "trima": "terima kasih", "trm": "terima", "trs": "terus", "trutama": "terutama", "ts": "penulis", "tst": "tahu sama tahu", "ttg": "tentang", "tuch": "tuh", "tuir": "tua", "tw": "tahu", "u": "kamu", "ud": "sudah", "udah": "sudah", "ujg": "ujung", "ul": "ulangan", "unyu": "lucu", "uplot": "unggah", "urang": "saya", "usah": "perlu", "utk": "untuk", "valas": "valuta asing", "w/": "dengan", "wadir": "wakil direktur", "wamil": "wajib militer", "warkop": "warung kopi", "warteg": "warung tegal", "wat": "buat", "wkt": "waktu", "wtf": "what the fuck", "xixixi": "tertawa", "ya": "iya", "yap": "iya", "yaudah": "ya sudah", "yawdah": "ya sudah", "yg": "yang", "yl": "yang lain", "yo": "iya", "yowes": "ya sudah", "yup": "iya", "7an": "tujuan", "ababil": "abg labil", "acc": "accord", "adlah": "adalah", "adoh": "aduh", "aha": "tertawa", "aing": "saya", "aja": "saja", "ajj": "saja", "aka": "dikenal juga sebagai", "akko": "aku", "akku": "aku", "akyu": "aku", "aljasa": "asal jadi saja", "ama": "sama", "ambl": "ambil", "anjir": "anjing", "ank": "anak", "ap": "apa", "apaan": "apa", "ape": "apa", "aplot": "unggah", "apva": "apa", "aqu": "aku", "asap": "sesegera mungkin", "aseek": "asyik", "asek": "asyik", "aseknya": "asyiknya", "asoy": "asyik", "astrojim": "astagfirullahaladzim", "ath": "kalau begitu", "atuh": "kalau begitu", "ava": "avatar", "aws": "awas", "ayang": "sayang", "ayok": "ayo", "bacot": "banyak bicara", "bales": "balas", "bangdes": "pembangunan desa", "bangkotan": "tua", "banpres": "bantuan presiden", "bansarkas": "bantuan sarana kesehatan", "bazis": "badan amal, zakat, infak, dan sedekah", "bcoz": "karena", "beb": "sayang", "bejibun": "banyak", "belom": "belum", "bener": "benar", "ber2": "berdua", "berdikari": "berdiri di atas kaki sendiri", "bet": "banget", "beti": "beda tipis", "beut": "banget", "bgd": "banget", "bgs": "bagus", "bhubu": "tidur", "bimbuluh": "bimbingan dan penyuluhan", "bisi": "kalau-kalau", "bkn": "bukan", "bl": "beli", "blg": "bilang", "blm": "belum", "bls": "balas", "bnchi": "benci", "bngung": "bingung", "bnyk": "banyak", "bohay": "badan aduhai", "bokep": "porno", "bokin": "pacar", "bole": "boleh", "bolot": "bodoh", "bonyok": "ayah ibu", "bpk": "bapak", "brb": "segera kembali", "brngkt": "berangkat", "brp": "berapa", "brur": "saudara laki-laki", "bsa": "bisa", "bsk": "besok", "bu_bu": "tidur", "bubarin": "bubarkan", "buber": "buka bersama", "bujubune": "luar biasa", "buser": "buru sergap", "bwhn": "bawahan", "byar": "bayar", "byr": "bayar", "c8": "chat", "cabut": "pergi", "caem": "cakep", "cama-cama": "sama-sama", "cangcut": "celana dalam", "cape": "capek", "caur": "jelek", "cekak": "tidak ada uang", "cekidot": "coba lihat", "cemplungin": "cemplungkan", "ceper": "pendek", "ceu": "kakak perempuan", "cewe": "cewek", "cibuk": "sibuk", "cin": "cinta", "ciye": "cie", "ckck": "ck", "clbk": "cinta lama bersemi kembali", "cmpr": "campur", "cnenk": "senang", "congor": "mulut", "cow": "cowok", "coz": "karena", "cpa": "siapa", "gokil": "gila", "gombal": "suka merayu", "gpl": "tidak pakai lama", "gpp": "tidak apa-apa", "gretong": "gratis", "gt": "begitu", "gtw": "tidak tahu", "gue": "saya", "guys": "teman-teman", "gws": "cepat sembuh", "haghaghag": "tertawa", "hakhak": "tertawa", "handak": "bahan peledak", "hansip": "pertahanan sipil", "hellow": "halo", "helow": "halo", "hi": "hai", "hlng": "hilang", "hnya": "hanya", "houm": "rumah", "hrs": "harus", "hubad": "hubungan angkatan darat", "hubla": "perhubungan laut", "huft": "mengeluh", "humas": "hubungan masyarakat", "idk": "saya tidak tahu", "ilfeel": "tidak suka", "imba": "jago sekali", "imoet": "imut", "info": "informasi", "itung": "hitung", "isengin": "bercanda", "iyala": "iya lah", "iyo": "iya", "jablay": "jarang dibelai", "jadul": "jaman dulu", "jancuk": "anjing", "jd": "jadi", "jdikan": "jadikan", "jg": "juga", "jgn": "jangan", "jijay": "jijik", "jkt": "jakarta", "jnj": "janji", "jth": "jatuh", "jurdil": "jujur adil", "jwb": "jawab", "ka": "kakak", "kabag": "kepala bagian", "kacian": "kasihan", "kadit": "kepala direktorat", "kaga": "tidak", "kaka": "kakak", "kamtib": "keamanan dan ketertiban", "kamuh": "kamu", "kamyu": "kamu", "kapt": "kapten", "kasat": "kepala satuan", "kasubbid": "kepala subbidang", "kau": "kamu", "kbar": "kabar", "kcian": "kasihan", "keburu": "terlanjur", "kedubes": "kedutaan besar", "kek": "seperti", "keknya": "kayaknya", "keliatan": "kelihatan", "keneh": "masih", "kepikiran": "terpikirkan", "kepo": "mau tahu urusan orang", "kere": "tidak punya uang", "kesian": "kasihan", "ketauan": "ketahuan", "keukeuh": "keras kepala", "khan": "kan", "kibus": "kaki busuk", "kk": "kakak", "klian": "kalian", "klo": "kalau", "kluarga": "keluarga", "klwrga": "keluarga", "kmari": "kemari", "kmpus": "kampus", "kn": "kan", "knl": "kenal", "knpa": "kenapa", "kog": "kok", "kompi": "komputer", "komtiong": "komunis Tiongkok", "konjen": "konsulat jenderal", "koq": "kok", "kpd": "kepada", "kptsan": "keputusan", "krik": "garing", "krn": "karena", "ktauan": "ketahuan", "ktny": "katanya", "kudu": "harus", "kuq": "kok", "ky": "seperti", "kykny": "kayanya", "laka": "kecelakaan", "lambreta": "lambat", "lansia": "lanjut usia", "lapas": "lembaga pemasyarakatan", "lbur": "libur", "lekong": "laki-laki", "lg": "lagi", "lgkp": "lengkap", "lht": "lihat", "linmas": "perlindungan masyarakat", "lmyan": "lumayan", "lngkp": "lengkap", "loch": "loh", "lol": "tertawa", "lom": "belum", "loupz": "cinta", "lowh": "kamu", "lu": "kamu", "luchu": "lucu", "luff": "cinta", "luph": "cinta", "lw": "kamu", "lwt": "lewat", "maaciw": "terima kasih", "mabes": "markas besar", "macem-macem": "macam-macam", "madesu": "masa depan suram", "maen": "main", "mahatma": "maju sehat bersama", "mak": "ibu", "makasih": "terima kasih", "malah": "bahkan", "malu2in": "memalukan", "mamz": "makan", "manies": "manis", "mantep": "mantap", "markus": "makelar kasus", "mba": "mbak", "mending": "lebih baik", "mgkn": "mungkin", "mhn": "mohon", "miker": "minuman keras", "milis": "mailing list", "mksd": "maksud", "mls": "malas", "mnt": "minta", "moge": "motor gede", "mokat": "mati", "mosok": "masa", "msh": "masih", "mskpn": "meskipun", "msng2": "masing-masing", "muahal": "mahal", "muker": "musyawarah kerja", "mumet": "pusing", "muna": "munafik", "munaslub": "musyawarah nasional luar biasa", "musda": "musyawarah daerah", "muup": "maaf", "muuv": "maaf", "nal": "kenal", "nangis": "menangis", "naon": "apa", "napol": "narapidana politik", "naq": "anak", "narsis": "bangga pada diri sendiri", "nax": "anak", "ndak": "tidak", "ndut": "gendut", "nekolim": "neokolonialisme", "nelfon": "menelepon", "ngabis2in": "menghabiskan", "ngakak": "tertawa", "ngambek": "marah", "ngampus": "pergi ke kampus", "ngantri": "mengantri", "ngapain": "sedang apa", "ngaruh": "berpengaruh", "ngawur": "berbicara sembarangan", "ngeceng": "kumpul bareng-bareng", "ngeh": "sadar", "ngekos": "tinggal di kos", "ngelamar": "melamar", "ngeliat": "melihat", "ngemeng": "bicara terus-terusan", "ngerti": "mengerti", "nggak": "tidak", "ngikut": "ikut", "nginep": "menginap", "ngisi": "mengisi", "ngmg": "bicara", "ngocol": "lucu", "ngomongin": "membicarakan", "ngumpul": "berkumpul", "ni": "ini", "nyasar": "tersesat", "nyariin": "mencari", "nyiapin": "mempersiapkan", "nyiram": "menyiram", "nyok": "ayo", "o/": "oleh", "ok": "ok", "priksa": "periksa", "pro": "profesional", "psn": "pesan", "psti": "pasti", "puanas": "panas", "qmo": "kamu", "qt": "kita", "rame": "ramai", "raskin": "rakyat miskin", "red": "redaksi", "reg": "register", "rejeki": "rezeki", "renstra": "rencana strategis", "reskrim": "reserse kriminal", "sni": "sini", "somse": "sombong sekali", "sorry": "maaf", "sosbud": "sosial-budaya", "sospol": "sosial-politik", "sowry": "maaf", "spd": "sepeda", "sprti": "seperti", "spy": "supaya", "stelah": "setelah", "subbag": "subbagian", "sumbangin": "sumbangkan", "sy": "saya", "syp": "siapa", "tabanas": "tabungan pembangunan nasional", "tar": "nanti", "taun": "tahun", "tawh": "tahu", "tdi": "tadi", "te2p": "tetap", "tekor": "rugi", "telkom": "telekomunikasi", "telp": "telepon", "temen2": "teman-teman", "tengok": "menjenguk", "terbitin": "terbitkan", "tgl": "tanggal", "thanks": "terima kasih", "thd": "terhadap", "thx": "terima kasih", "tipi": "TV", "tkg": "tukang", "tll": "terlalu", "tlpn": "telepon", "tman": "teman", "tmbh": "tambah", "tmn2": "teman-teman", "tmph": "tumpah", "tnda": "tanda", "tnh": "tanah", "togel": "toto gelap", "tp": "tapi", "tq": "terima kasih", "trgntg": "tergantung", "trims": "terima kasih", "cb": "coba", "y": "ya", "munfik": "munafik", "reklamuk": "reklamasi", "sma": "sama", "tren": "trend", "ngehe": "kesal", "mz": "mas", "analisise": "analisis", "sadaar": "sadar", "sept": "september", "nmenarik": "menarik", "zonk": "bodoh", "rights": "benar", "simiskin": "miskin", "ngumpet": "sembunyi", "hardcore": "keras", "akhirx": "akhirnya", "solve": "solusi", "watuk": "batuk", "ngebully": "intimidasi", "masy": "masyarakat", "still": "masih", "tauk": "tahu", "mbual": "bual", "tioghoa": "tionghoa", "ngentotin": "senggama", "kentot": "senggama", "faktakta": "fakta", "sohib": "teman", "rubahnn": "rubah", "trlalu": "terlalu", "nyela": "cela", "heters": "pembenci", "nyembah": "sembah", "most": "paling", "ikon": "lambang", "light": "terang", "pndukung": "pendukung", "setting": "atur", "seting": "akting", "next": "lanjut", "waspadalah": "waspada", "gantengsaya": "ganteng", "parte": "partai", "nyerang": "serang", "nipu": "tipu", "ktipu": "tipu", "jentelmen": "berani", "buangbuang": "buang", "tsangka": "tersangka", "kurng": "kurang", "ista": "nista", "less": "kurang", "koar": "teriak", "paranoid": "takut", "problem": "masalah", "tahi": "kotoran", "tirani": "tiran", "tilep": "tilap", "happy": "bahagia", "tak": "tidak", "penertiban": "tertib", "uasai": "kuasa", "mnolak": "tolak", "trending": "trend", "taik": "tahi", "wkwkkw": "tertawa", "ahokncc": "ahok", "istaa": "nista", "benarjujur": "jujur", "mgkin": "mungkin"}
nlp = spacy.blank('id') 

def tokenisasi(text): #TOKENISASI
    temp=[]
    for berita_2 in text:    
        spacy_id = nlp(berita_2)
        token_kata = [token.text for token in spacy_id]
        temp.append(token_kata)
    return (temp)

def stopword(text):
    temp=[]
    temp1=[]
    for i in range(len(text)):
        temp=[w for w in text[i] if not w in STOP_WORDS]
        temp1.append(temp)
    return temp1
        
def stemming(text):
    temp=[]
    temp1=[]
    for i in text:
        for j in i :
            temp.append(stemmer.stem(j))
        temp1.append(temp)
        temp=[]
    return temp1

def filtering(text):
    for i in range(len(text)):
        j=0
        text[i]=[x for x in text[i] if x]
        text[i]=[x for x in text[i] if len(x) < 32] #menghapus link
        for item in text[i]:
            text[i][j]=filtering2(item)
            j=j+1
    return text

    
def filtering2(text): 
    #remove link web
    text=re.sub(r"\S*https?:\S*", '', text) 
    #remove simbol
    text=re.sub(r"[.,:;+!\-_<^/=?\"'\(\)\d\*]", " ", text)
    #remove space
    text=re.sub(r"[' ']","",text)
    return text
    

def normalisasi(text):
    temp=[]
    for i in range(len(text)):
        temp.append([])
        for t in text[i]:
            if ((t != '') or (t!="")):
                temp[i].append(normal_list[t] if t in normal_list else t)
    return temp

nlp = spacy.blank('id')

#memasukkan semua berita pada satu variabel
berita=[]
for file in os.listdir(path): 
    if os.path.isfile(os.path.join(path, file)):
        file_path=f"{path}\{file}"
        with open(file_path,'r') as t:
            berita.append(t.read().lower()) #case-folding


#mentokenisasi berita
berita=tokenisasi(berita)

#eliminasi stopword
berita=stopword(berita)

#stemming
berita=stemming(berita)

#filtering
berita=filtering(berita)

#normalisasi        
berita=normalisasi(berita)

print(berita)




[['wilayah', 'bebas', 'covid', 'cek', 'kabupaten', 'kota', 'zona', 'hijau', 'baru', 'jakarta', 'perintah', 'rencana', 'terap', 'laku', 'batas', 'giat', 'masyarakat', 'ppkm', 'level', 'hitung', 'desember', 'januari', 'menteri', 'sehat', 'ri', 'bijak', 'ppkm', 'level', 'tahap', 'kaji', 'direktur', 'cegah', 'kendali', 'sakit', 'tular', 'langsung', 'ppml', 'kemenkes', 'ri', 'dr', 'siti', 'nadia', 'tarmizi', 'ppkm', 'level', 'terap', 'covid', 'signifikan', 'picu', 'tingkat', 'mobilitas', 'longgar', 'protokol', 'sehat'], ['vaksin', 'covid', 'rutin', 'gantung', 'jelas', 'jakarta', 'beri', 'booster', 'dosis', 'tiga', 'vaksin', 'covid', 'indonesia', 'rencana', 'januari', 'lantas', 'ada', 'vaksinasi', 'covid', 'vaksinasi', 'influenza', 'ketua', 'satgas', 'covid', 'ikat', 'dokter', 'indonesia', 'idi', 'prof', 'zubairi', 'djoerban', 'pasti', 'kait', 'turut', 'vaksin', 'covid', 'booster', 'vaksinasi', 'covid'], ['ri', 'suntik', 'booster', 'ampuh', 'lawan', 'varian', 'delta', 'cs', 'jakarta', 'pakar

Penjelasan Preprocessing tahapannya

1. Import kebutuhan package
2. Siapkan path pada folder berita
3. Teknik preprocessing yang digunakan :

3.1 Untuk stopword saya import dari spacy 

3.2 Untuk normalisasi kata saya ambil dari github(https://github.com/louisowen6/NLP_bahasa_resources/blob/master/combined_slang_words.txt)

3.3 Untuk Tokenisasi saya bikin fungsinya

3.4 Fungsi filtering untuk menghapus angka dan simbol

3.5 Stemming menggunakan Sastrawi

3.6 Case-folding digunakan ketika memasukkan isi berita.txt ke list berita
    
4. Masukkan berita.txt dalam file berita yang sudah didapatkan path directory ke dalam List berita (dalam memasukkan berita.txt term sudah dalam keadaan case-folding)
5. List berita di tokenisasi menggunakan fungsi tokenisasi(berita)
6. Setelah itu dilakukan eliminasi stopword dan stemming dengan fungsi stopword dan stemming
7. Setelah itu berita difilter menggunakan fungsi filter dan menghapus :

7.1 Term yang berisi spasi [' '] 

7.2 Menghapus link

7.3 Menghapus simbol dan angka

8. Setelah itu berita di normalisasi dengan dictionary dari github dan menghilangkan term kosong [''] 
9. Setelah itu di print berita 
10. berita merupakan list yang tiap indeksnya list yang berisi term. Term ini sudah di preprocessing dengan tahapan di atas


# Inverted Index


In [37]:
inverted_index = {}

for i in range(len(berita)):
    for item in berita[i]:
        if item not in inverted_index:
            inverted_index[item] = []
        if (item in inverted_index) and ((i+1) not in inverted_index[item]):
            inverted_index[item].append(i+1)
print(inverted_index)

{'wilayah': [1, 5], 'bebas': [1], 'covid': [1, 2, 3, 4, 5], 'cek': [1], 'kabupaten': [1], 'kota': [1], 'zona': [1], 'hijau': [1], 'baru': [1, 3, 4], 'jakarta': [1, 2, 3, 4, 5], 'perintah': [1], 'rencana': [1, 2], 'terap': [1], 'laku': [1], 'batas': [1], 'giat': [1], 'masyarakat': [1], 'ppkm': [1], 'level': [1], 'hitung': [1], 'desember': [1], 'januari': [1, 2], 'menteri': [1], 'sehat': [1], 'ri': [1, 3, 4], 'bijak': [1], 'tahap': [1], 'kaji': [1], 'direktur': [1], 'cegah': [1], 'kendali': [1], 'sakit': [1], 'tular': [1], 'langsung': [1], 'ppml': [1], 'kemenkes': [1, 4], 'dr': [1, 5], 'siti': [1], 'nadia': [1], 'tarmizi': [1], 'signifikan': [1, 4], 'picu': [1], 'tingkat': [1, 4], 'mobilitas': [1], 'longgar': [1], 'protokol': [1], 'vaksin': [2, 3], 'rutin': [2], 'gantung': [2], 'jelas': [2], 'beri': [2], 'booster': [2, 3], 'dosis': [2, 3], 'tiga': [2, 3], 'indonesia': [2, 3, 4], 'lantas': [2], 'ada': [2], 'vaksinasi': [2], 'influenza': [2], 'ketua': [2, 3], 'satgas': [2, 3], 'ikat': [2, 

Penjelasan Inverted Index 

1. Membuat dictionary kosong bernama inveted_index
2. Buat perulangan (i) sebanyak jumlah list dalam list berita (untuk berita yang dipakai ada 5)
3. Perulangan (i) ini dipakai untuk menelusuri tiap index dalam list berita.
4. bentuk index dalam berita yaitu list yang berisi term yang sudah di preprocessing (list dalam list)
5. Kita cek setiap term:

5.1 apabila term tidak ditemukan dalam key dictionary maka bikin key baru dengan value i + 1 .

5.2 apabila term yang ditemukan ada di key dictionary dan memiliki value yang sama maka tidak dilakukan apa-apa, namun jika memiliki value yang beda maka ditambahkan value baru i+1 (agar tidak terjadi pencatatan dobel letak term pada list)

6. Setelah itu cetak inverted_index
7. Hasil dari inverted_index yaitu dictionary dengan key=term dan valuenya=list yang berisi letak term dalam berita contohnya term covid berada di dokumen ke 1,2,3,4, dan 5


# Penggunaan Boolean Retrievel Model

In [40]:
print(process_query("corona",len(berita),inverted_index))


print(process_query("covid",len(berita),inverted_index))

print(process_query("vaksin",len(berita),inverted_index))

print(process_query("corona OR covid",len(berita),inverted_index))

print(process_query("vaksin AND corona",len(berita),inverted_index))

print(process_query("vaksin AND (corona OR covid)",len(berita),inverted_index))


['corona']
[3, 5]
['covid']
[1, 2, 3, 4, 5]
['vaksin']
[2, 3]
['corona', 'OR', 'covid']
[1, 2, 3, 4, 5]
['vaksin', 'AND', 'corona']
[3]
['vaksin', 'AND', '(', 'corona', 'OR', 'covid', ')']
[2, 3]


Penjelasan Boolean Retrievel Model

1. Dipanggil print(process_query(query,n_docs, inverted_index) query merupakan term yang dicari, n_docs merupakan jumlah indeks dalam list berita, dan inverted_index merupakan dictionary yang mencatat letak term dalam suatu dokumen.

2. Setelah itu dalam tahapan process_query :

2.1 Dilakukan tokenisasi pada query dan di cetak

2.2 dipanggil fungsi parse_query yang bertujuan untuk mengembalikan query dalam bentuk urutan agar diproses dengan urutan yang benar. Contoh : query="vaksin AND (corona OR covid)", pengembaliannya menjadi ['vaksin', 'corona', 'covid', 'OR', 'AND']. Urutan ini memisahkan term dan operasi. List urutan ini disimpan dalam variabel postfix_queue

2.3 Setelah itu dilakukan perulangan sebanyak postfix_queue

2.4 Dalam perulangan ini dipop postfix_queue index awal. Jika, index berisi term maka dicek apakah term tersebut terdapat didalam key dictionary. Jika iya maka valuenya diappend dalam list operator_stack, jika tidak do nothing

2.5 Jika, index awal postfix_queue berisi operasi maka dilakukan operasi pada operator_stack index ke 2 akhir. Contoh :
operator_stack=[[1,2],[2,3],[3,4]] Ketika index awal postfix_queue berisi operasi AND maka dilakukan operasi [2,3] AND [3,4] Maka operator_stack terbaru menjadi [[1,2],[3]] Jika postfix_queue selanjutnya berisi OR maka dilakukan operasi [1,2] OR [3]. Dan hasil terakhir operator_stack [1,2,3].

2.6 Setelah perulangan postfix_queue selesai didapatkan operator_stack, dan dicek berapa banyak index operator_stack. Jika banyak indeks tidak sama dengan 1 cetak eror dan pengembalian None. Jika banyak indeks sama dengan 1 dilakukan pengembalian pop pada operator_stack.

3. Cetak nilai operator_stack



